In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
import json
from mpcontribs.io.archieml.mpfile import MPFile
from mpcontribs.io.core.recdict import RecursiveDict
from mpcontribs.rest.adapter import ContributionMongoAdapter
from mpcontribs.builders import MPContributionsBuilder
from mpcontribs.rest.rester import MPContribsRester
from mpcontribs.utils import submit_mpfile
print 'all imported'

In [ ]:
cma = ContributionMongoAdapter()
contributor = 'Patrick Huck <phuck@lbl.gov>'
#mpfile = MPFile.from_file('atn_Ni20Fe80Pt10_archieml.txt')
mpfile = MPFile.from_file('/tmp/mpfile_out.txt')
#print json.dumps(mpfile.document, indent=4)
mpfile_singles = mpfile.split()
build_doc = RecursiveDict()

axes, ov_data = set(), dict()
for idx,mpfile_single in enumerate(mpfile_singles):
    mp_cat_id = mpfile_single.document.keys()[0]
    print idx, mp_cat_id
    doc = cma.submit_contribution(mpfile_single, contributor)
    cid = str(doc['_id'])
    mpfile_single.insert_id(mp_cat_id, cid)
    mcb = MPContributionsBuilder(doc)
    single_build_doc = mcb.build(contributor, cid)
    scope, local_axes = [], set()
    tree_data = single_build_doc[3]['tree_data']
    for k,v in tree_data.iterate():
        if v is None:
            scope = scope[:k[0]]
            scope.append(k[1])
        else:
            try:
                if k[0] == len(scope):
                    scope.append(k[1])
                else:
                    scope[-1] = k[1]
                vf = float(v) # trigger exception
                scope_str = '.'.join(scope)
                if idx == 0:
                    axes.add(scope_str)
                    ov_data[scope_str] = {cid: (vf, mp_cat_id)}
                else:
                    local_axes.add(scope_str)
                    ov_data[scope_str][cid] = (vf, mp_cat_id)
            except:
                pass
    if idx > 0:
        axes.intersection_update(local_axes)
    #break
    #build_doc.rec_update(single_build_doc)
for k in ov_data:
    if k not in axes:
        ov_data.pop(k)
print ov_data

#print json.dumps(build_doc, indent=4)

In [ ]:
#mpfile = MPFile.from_string('{hello_world}\nkey: value\n\n[+mp-30]\nx,y\n1,0\n2,1\n3,2\n')
#mpfile = MPFile.from_file('atn_Ni20Fe80Pt10_archieml.txt')
#mpfile = MPFile.from_file('/Users/patrick/Downloads/mpfile_test.txt')
mpfile = MPFile.from_file('test_archieml.txt')
mpfile_test = MPFile.from_string(mpfile.get_string())
if mpfile.document != mpfile_test.document:  
    mpfile_json_lines = json.dumps(mpfile.document, indent=4).split('\n')
    mpfile_test_json_lines = json.dumps(mpfile_test.document, indent=4).split('\n')
    try:
        for a, b in zip(mpfile_json_lines, mpfile_test_json_lines):
            if a != b:
                raise Exception('{} <=====> {}'.format(a.strip(), b.strip()))
    except Exception as ex:
        print 'dicts not equal:'
        print str(ex)
        print json.dumps(mpfile.document, indent=4)
        print json.dumps(mpfile_test.document, indent=4)
else:
        print 'dicts equal'

#print '================ rec_update ================='
#mpfile.document.rec_update(
#    other={'general':{'Experiment':{'Preparation':{'Description':'hello', 'a':3}}}}, overwrite=True)
#print json.dumps(mpfile.document, indent=4)

In [ ]:
build_doc = submit_mpfile('test_files/atn_Ni20Fe80Pt10_archieml.txt', fmt='archieml')
print json.dumps(build_doc, indent=4)

In [ ]:
mpfile = MPFile.from_file('test_archieml.txt')
mpfile_singles = mpfile.split()

with MPContribsRester('r1bnxivUyJ26cgho', endpoint='http://localhost:8000/mpcontribs/rest') as mpr:
    print mpr.check_contributor()
    for mpfile_single in mpfile_singles:
        print mpr.submit_contribution(mpfile_single, 'archieml')
        break